# CNN with a Trainable Embedding Layer

In [1]:
%matplotlib inline

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [2]:
import ktrain
from ktrain import text

import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
from gensim import utils

## Compute custom Word2Vec embeddings

Load the full corpus.

In [3]:
import pandas as pd
full_corpus = pd.read_csv('../data/ICAAD_FIJI.csv')
full_corpus_text = full_corpus.contents
dummy_labels = full_corpus.sentencing

Instantiate the preprocessor

In [4]:
NUM_WORDS = 33000
MAXLEN = 5000
DIMS = 300
NGRAM_RANGE = 1

Preprocess the full corpus.  Note that we need to pass a dummy vector of lables.  These don't matter.

In [5]:
preproc = ktrain.text.preprocessor.StandardTextPreprocessor(max_features=NUM_WORDS, maxlen=MAXLEN, class_names=[0,1])

In [6]:
X, dummy_y = preproc.preprocess_train(full_corpus_text, dummy_labels)

language: en
Word Counts: 132011
Nrows: 13384
13384 train sequences
train sequence lengths:
	mean : 2218
	95percentile : 5989
	99percentile : 11259
x_train shape: (13384,5000)
y_train shape: (13384, 2)
Is Multi-Label? False


In [7]:
X = list(X)

Convert the tokenized documents back into text in preparation for running through Word2Vec.  This step may not be strictly necessary.

In [8]:
X_text = [preproc.undo(doc).split(' ') for doc in X]

In [9]:
def build_model(corpus, dims=DIMS, workers=8):
    WV = Word2Vec(sentences=corpus, size=dims, workers=8)
    vocab = WV.wv.index2word
    vocab_len = len(vocab)
    embeddings = np.array([WV.wv.get_vector(word) for word in vocab])
    text_to_token = {word: i for word, i in zip(vocab, range(vocab_len))}
    token_to_text = {i: word for word, i in zip(vocab, range(vocab_len))}
    model = {'embeddings': embeddings,
             'text_to_token': text_to_token,
             'token_to_text': token_to_text,
             'vocab': vocab,
             'vocab_len': len(vocab)}
    return model

In [11]:
def embed_word(word, wv_model, dims):
    if word in wv_model['vocab']:
        token = wv_model['text_to_token'][word]
        return wv_model['embeddings'][token]
    else:
        return np.zeros(dims)

In [14]:
def build_embeddings(preproc, wv_model, dims):
    preproc_vocab = preproc.undo(range(NUM_WORDS)).split(' ')
    Embeddings = [embed_word(word, wv_model, DIMS) for word in preproc_vocab]
    Embeddings = np.stack(Embeddings)
    return(Embeddings)

In [16]:
wv_model = build_model(X_text, DIMS)
Embeddings = build_embeddings(preproc, wv_model, DIMS)

In [17]:
Train = pd.read_csv('../data/train.csv')
x_train = Train.cleaned_contents
y_train = Train.Discrimination_Label

Test = pd.read_csv('../data/test.csv')
x_test = Test.cleaned_contents
y_test = Test.Discrimination_Label

In [18]:
train = preproc.preprocess_test(x_train, y_train)
test = preproc.preprocess_test(x_test, y_test)

647 test sequences
test sequence lengths:
	mean : 1515
	95percentile : 3968
	99percentile : 7513
x_test shape: (647,5000)
y_test shape: (647, 2)
162 test sequences
test sequence lengths:
	mean : 1419
	95percentile : 3336
	99percentile : 7205
x_test shape: (162,5000)
y_test shape: (162, 2)


## Build the model

In [19]:
def _build_cnn(maxlen, max_features, embed_dim, filters, kernels,
               dropout=0.1, density=64,
               loss_func='categorical_crossentropy',
               activation = 'softmax', metrics=['accuracy'],
               verbose=1, optimizer='adam'):
    
    embedding_matrix = np.ones((max_features, 1))
    embedding_matrix[0] = 0

    # set up the model
    inp = tf.keras.layers.Input(shape=(maxlen,))
    x = tf.keras.layers.Embedding(max_features, embed_dim, input_length=maxlen, 
                                  trainable=False)(inp)
    x0 = tf.keras.layers.Conv1D(filters=filters,
                               kernel_size=kernels[0],
                               activation='relu')(x)
    x0 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[0] + 1)(x0)

    x1 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[1],
                                activation='relu')(x)
    x1 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[1] + 1)(x1)
    
    x2 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[2],
                                activation='relu')(x)
    x2 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[2] + 1)(x2)
    
    x3 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[3],
                                activation='relu')(x)
    x3 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[3] + 1)(x3)

    x4 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[4],
                                activation='relu')(x)
    x4 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[4] + 1)(x4)

    x = tf.keras.layers.concatenate([x0, x1, x2, x3, x4])

    x = tf.keras.layers.Dense(density, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(2, activation=activation)(x)
    model = tf.keras.Model(inputs=inp, outputs=outputs)
    model.compile(loss=loss_func,
                  optimizer=optimizer,
                  metrics=metrics)
    return model

In [ ]:
model = _build_cnn(MAXLEN, len(Embeddings), DIMS, filters=64, kernels=[2, 3, 4, 5, 6], dropout=0.4, density=32)
learner = ktrain.get_learner(model, train_data=train, val_data=test)

In [ ]:
learner.model.summary()

## Add the pre-trained embeddings

In [ ]:
learner.model.layers[1].set_weights([Embeddings])

In [ ]:
learner.model.weights[0].numpy()[0]

In [ ]:
Embeddings

## Define and train the model

### Find a good initial learning rate

This is a method that was developed at the Naval Research Laboratory.  It's been promoted by Jeremy Howard.

In [ ]:
learner.lr_find(show_plot=True)

### Train

In [ ]:
learner.autofit(10e-4, early_stopping=5)

## Examine results

In [ ]:
learner.validate(class_names=preproc.get_classes())

## Explain a prediction

In [ ]:
import pandas as pd
X_test = pd.read_csv('../data/test.csv')['cleaned_contents']

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=preproc)

In [ ]:
i = 9
print(predictor.predict(x_test[i]), y_test[i])

In [ ]:
predictor.explain(x_test[i])

In [ ]:
learner.evaluate(test)[1,1]

In [ ]:
x_test.shape

In [ ]:
f1_best = 0.1
for width in [32, 64, 128, 256]:
    for filters in [16, 32, 64]:
        for dropout in [0.1, 0.2, 0.3, 0.4]:
            f1s = []
            for _ in range(5):
                model = _build_cnn(MAXLEN, len(Embeddings), DIMS, filters=filters, kernels=[2, 3, 4, 5, 6], dropout=dropout, density=width)
                learner = ktrain.get_learner(model, train_data=train, val_data=test)
                learner.model.layers[1].set_weights([Embeddings])
                print(learner.model.weights[0])
                learner.autofit(10e-4)
                c_matrix = learner.evaluate(test)
                TP = c_matrix[1,1]
                FN = c_matrix[0,1]
                FP = c_matrix[1,0]
                F1 = 2 * TP / (2 * TP + FN + FP)
                f1s.append(F1)
                if F1 > f1_best:
                    print('------------------------------------------------------------------------')
                    print(str(F1) + ' is better than ' + str(f1_best))
                    f1_best = F1
                    print('New f1_best = ' + str(f1_best))
                    tf.keras.models.save_model(model, filepath='./cnn_Ioya2Vec_model')
                    predictor = ktrain.get_predictor(model, preproc)
                    predictor.save('cnn_Ioya2Vec_predictor')
            outstring = str(width) + ',' + str(filters) + ',' + str(dropout) + ',' + str(np.array(f1s).mean()) + '\n'
            print(outstring)
            with open('results_updated.csv', 'a') as f:
                f.write(outstring)

<tf.Variable 'embedding_36/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001.

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:41: UserWarning: Could not save original model weights
  warnings.warn('Could not save original model weights')


<tf.Variable 'embedding_156/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:41: UserWarning: Could not save original model weights
  warnings.warn('Could not save original model weights')


<tf.Variable 'embedding_157/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001

...

<tf.Variable 'embedding_208/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:41: UserWarning: Could not save original model weights
  warnings.warn('Could not save original model weights')


<tf.Variable 'embedding_209/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:41: UserWarning: Could not save original model weights
  warnings.warn('Could not save original model weights')


<tf.Variable 'embedding_210/embeddings:0' shape=(32999, 300) dtype=float32, numpy=
array([[ 5.9036863e-01,  9.0349948e-01,  1.2093734e+00, ...,
         4.5612368e-01,  1.8838350e-01, -1.9618142e+00],
       [-5.8652055e-01,  2.9772940e-01,  3.1020391e-01, ...,
         1.1321809e+00,  8.5766238e-01, -2.0014069e+00],
       [ 5.9271681e-01,  8.1031388e-01, -8.5649437e-01, ...,
         1.3010132e+00,  4.4007072e-01, -3.6066253e+00],
       ...,
       [ 2.9657234e-02,  2.2038482e-02, -5.6051318e-02, ...,
         6.9732770e-02, -1.1526353e-01,  2.1006858e-02],
       [ 8.8998429e-02,  8.6988613e-02, -2.5797749e-02, ...,
         8.0753088e-02, -7.5066857e-02,  1.6337280e-01],
       [ 2.5559446e-02,  2.3548608e-03,  6.6786364e-02, ...,
         9.2198504e-03, -5.5137150e-02,  1.2658484e-01]], dtype=float32)>
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001

In [ ]:
os.getcwd()

In [ ]:
outstring